The program asks a user to enter a **job title** and **location** of interest and scrapes relavant data from [a job posting website](www.indeed.com). Using NLTK, it removes stopwords and count the splited words. Then it asks a user to enter specific set of skills(separated by space) and return the sorted list of **skills** with number of occurence.

The goal for the small program is to help users to find out which skills and talents are mostly desired by employees. 

For example, if you want to know if python is more demanded than R for 'data analyst' position, you can figure it out yourself as below.

In [1]:
import requests # for accesing web page
from bs4 import BeautifulSoup # for pulling data out of html
import pandas as pd # for general working with data
# from nltk import word_tokenize # text mining / analysis
from collections import Counter
from nltk.corpus import stopwords
import re #regex

In [2]:
# A function take jobtitle and location as arguments and return correct url for web scraping purpose
def searchquery(jobtitle, location):
    title = jobtitle.replace(' ', '+')
    loc = location.replace(' ', '+')
    url = 'http://www.indeed.com/jobs?q=%22'+ title +'%22&radius=50&limit=50&l='+loc
    return url

# A function to take job list's url as an input and return dataframe with job titles and url to job post
def collect_job_list(url):
    # create empty list
    jobtitle, hreflink = [], []
    
    # get contents from the web
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    # find the page number
    x = soup.findAll('div', {'id': 'searchCount'})[0].text.replace(',', '')
    pageN = int(x[x.find('of ')+3:])
    
    # iterate over page number    
    for i in range(0, pageN, 50):
        joblisturl = url + '&start=' + str(i)
        r = requests.get(joblisturl)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # iterate over each listed job post on search result to obtain job title and link
        for data in soup.findAll('a', {'data-tn-element': 'jobTitle'}):
            if 'clk?jk=' in data.get('href'):
                hreflink.append(data.get('href'))
                jobtitle.append(data.text)
    df = pd.DataFrame({'title': jobtitle, 'link': hreflink})
    return df


# convert the href link data in dataframe to proper url
def properurl(link):
    joburl = 'http://www.indeed.com/viewjob?jk=' +\
            link[link.find('clk?jk=')+len('clk?jk='):link.find('&fccid')]
    return joburl
            

# A function to take job posting's url as an input, mine text data from selected job post. 
# and return the text from the post.
def collect_job_data(joblink_list):
    jobdesc = []
    #iterate over href link in data frame
    for i in range(0, len(joblink_list)):
        joburl = properurl(joblink_list[i])
        
        #extracting text data from selected job posting        
        r = requests.get(joburl)
        soup = BeautifulSoup(r.content, 'html.parser')
        desc = ''.join(soup.findAll('td', {'class': 'snip'})[0].text)
        desc = re.sub('[^A-Za-z0-9&]+', ' ', desc)
        jobdesc.append(desc[:desc.find('ago')].replace('\n', ' ').lower())
    return jobdesc

# A function to take str as input, split the str and count the words
def countword(text):
    #removing stopwords from the data
    stop = stopwords.words('english')   
    
    nostopword = ' '.join([word for word in text.split() if word not in stop])
    #create word count list
    count = Counter(nostopword.split())
    return count

# A function to take words as input and return the list of counts for the words of interest.
def sortlist(words, countlist):
    result= []  
    for word in words.split():
        result.append([x for x in countlist if word in x])
    return result

In [3]:
# prompt a user to input job title and location of interest
jobtitle = input('Please enter the "job title" of interest:  ') 
location = input('Please enter the "location" of interest:  ')

# using the 'collect_job_list' funciton, assign data frame to a variable 'df' 
df = collect_job_list(searchquery(jobtitle, location))

# remove duplicated data
df = df[df.duplicated('link') == False]

# add column of text that contains job description to the dataframe
df['text'] = collect_job_data(list(df.link))

# Clean data with text whose length is less than 100
for index, row in df.iterrows():
    if(len(row.text) < 100):
        row.text = ''
df = df[df.text != '']

# iterate over the dataframe to split the texts into words
for index, row in df.iterrows():
    row.text = countword(row.text)

# count the occurence of words
totalcount = Counter()
for index, row in df.iterrows():
    totalcount += row.text

# reorder the list of count in descending order
result = totalcount.most_common()

# overview of the most frequent words
print("Number of open position: "+str(len(df)))
print("\nThe 100 most frequent words are as below:\n")
print(result[:100])

Please enter the "job title" of interest:  data analyst
Please enter the "location" of interest:  california
Number of open position: 528

The 100 most frequent words are as below:

[('data', 4658), ('experience', 1964), ('business', 1565), ('skills', 1060), ('work', 1051), ('analysis', 938), ('team', 938), ('ability', 849), ('analyst', 819), ('management', 665), ('support', 608), ('strong', 589), ('years', 578), ('knowledge', 546), ('sql', 539), ('analytics', 538), ('reporting', 537), ('required', 508), ('requirements', 489), ('tools', 488), ('information', 487), ('days', 474), ('working', 471), ('development', 457), ('product', 449), ('reports', 446), ('quality', 411), ('including', 411), ('systems', 392), ('technical', 389), ('develop', 389), ('related', 388), ('position', 385), ('job', 379), ('environment', 370), ('customer', 362), ('degree', 361), ('using', 349), ('preferred', 348), ('company', 346), ('provide', 343), ('&', 342), ('analytical', 340), ('insights', 336), ('communica

In [4]:
# prompt a user to sort the list with words of interest
intwords = input('Enter the words of interst separated by space\n')
sortlist(intwords, result)

Enter the words of interst separated by space
sql excel r python hadoop


[[('sql', 539)],
 [('excel', 298)],
 [('r', 108)],
 [('python', 111)],
 [('hadoop', 62)]]

The most desired skill for 'data analyst' is SQL, followed by excel, which is ubiquitous tool for any industry.

R and Python are almost equivalently demanded by employers.

Hadoop is less demanded than other skills for 'data analyst' position.